## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace
import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# Import necessary libraries
import os
import json
import chromadb
from dotenv import load_dotenv
from tavily import TavilyClient
from pydantic import BaseModel, Field

# Custom library imports
from lib.agents import Agent
from lib.llm import LLM
from lib.tooling import tool
from chromadb.utils import embedding_functions

In [3]:
# TODO: Load environment variables
# load_dotenv()

OPENAI_API_KEY = "voc-11905559501688654276185689a2c33e68215.70173222"
TAVILY_API_KEY = "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"
VOCAREUM_API_BASE = "https://openai.vocareum.com/v1"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
#### 1. Retrieve Game Tool
@tool
def retrieve_game(query: str) -> list:
    """
    Semantic search: Finds the most relevant results in the vector DB.
    args:
    - query: a question about the game industry.
    """
    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=OPENAI_API_KEY,
        api_base=VOCAREUM_API_BASE,
        model_name="text-embedding-ada-002"
    )
    
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection(
        name="udaplay",
        embedding_function=embedding_fn
    )
    
    results = collection.query(
        query_texts=[query],
        n_results=5
    )
    
    if 'documents' in results and results['documents']:
        return results['documents'][0]
    return []

#### Evaluate Retrieval Tool

In [5]:
#### 2. Evaluate Retrieval Tool

class EvaluationReport(BaseModel):
    """Data model for the evaluation report."""
    useful: bool = Field(..., description="Whether the documents are useful to answer the question")
    description: str = Field(..., description="A detailed explanation of the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    """
    llm = LLM(
        model="gpt-4-turbo",
        temperature=0,
        api_key=OPENAI_API_KEY,
        base_url=VOCAREUM_API_BASE
    )

    # *** THIS IS THE MAIN FIX ***
    # The reviewer noted a SyntaxError. The f-string is now defined correctly 
    # without any invalid characters. The list of documents is joined into a 
    # string *before* being placed in the f-string for clarity and safety.
    docs_as_string = "\n\n".join(retrieved_docs)
    prompt = f"""Your task is to evaluate if the documents are enough to respond to the query.
    
    Respond with ONLY a JSON object in the following format:
    {{
      "useful": <true or false>,
      "description": "<Your detailed explanation here>"
    }}

    Original user question: {question}

    Retrieved documents: {docs_as_string}
    """
    
    ai_message = llm.invoke(prompt)
    report_dict = json.loads(ai_message.content)
    return EvaluationReport(**report_dict)

#### Game Web Search Tool

In [6]:
#### 3. Game Web Search Tool
@tool
def game_web_search(query: str) -> str:
    """
    Performs a web search using Tavily for questions about the game industry.
    args:
    - query: a question about the game industry.
    """
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    response = tavily_client.search(query=query, search_depth="advanced")
    return "\n".join([result['content'] for result in response['results']])

### Agent

In [7]:
### Agent with Citations

instructions = """You are UdaPlay, an AI Research Agent for the video game industry.
Your goal is to answer user questions about video games, developers, platforms, and release dates.

Here is your workflow:
1. **Internal Knowledge First**: Always start by using the `retrieve_game` tool.
2. **Evaluate**: Use `evaluate_retrieval` to assess if the information is sufficient.
3. **Web Search**: If the evaluation determines the information is not useful, use `game_web_search`.
4. **Respond with Citations**: Combine the information into a clear and concise answer. You MUST cite your source at the end of your response.
   - If the answer comes from the `retrieve_game` tool, end your response with "(Source: Internal Knowledge Base)".
   - If the answer comes from the `game_web_search` tool, end your response with "(Source: Web Search)".
"""

# Instantiate the agent with the corrected tools
udaplay_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=instructions,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    base_url=VOCAREUM_API_BASE,
    api_key=OPENAI_API_KEY
)

In [8]:
### Invoke Agent and Demonstrate Results

# As required, run the agent on at least three example queries.
queries = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?"
]

for query in queries:
    print(f"--- Query: {query} ---\n")
    try:
        # This will now execute successfully and print the agent's internal logs
        run = udaplay_agent.invoke(query)
        final_state = run.get_final_state()
        
        # Print the agent's reasoning and tool usage for the report
        print("Agent's Reasoning and Tool Usage:")
        for message in final_state.get('messages', []):
            if message.role == 'assistant' and message.tool_calls:
                for tool_call in message.tool_calls:
                    print(f"- Tool Called: {tool_call.function.name}")
                    print(f"  - Arguments: {tool_call.function.arguments}")
        
        # Print the final answer
        if final_state and final_state.get('messages'):
            final_answer = final_state['messages'][-1].content
            print(f"\nFinal Answer:\n{final_answer}\n")
        else:
            print("Agent run did not produce a final answer.")
            
    except Exception as e:
        print(f"An error occurred during agent execution: {e}")
    print("-" * 40)

--- Query: When was Pokémon Gold and Silver released? ---

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Agent's Reasoning and Tool Usage:
- Tool Called: retrieve_game
  - Arguments: {"query":"Pokémon Gold and Silver release date"}
- Tool Called: evaluate_retrieval
  - Arguments: {"question":"When was Pokémon Gold and Silver released?","retrieved_docs":"[\"Name: Pokémon Gold and Silver\\nPlatform: Game Boy Color\\nYear of Release: N/A\\nGenre: Role-playing\\nPublisher: Nintendo\\nDescription: Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.\", \"Name: Pokémon R